In [1]:
import mwclient
import time

site = mwclient.Site("en.wikipedia.org")
page = site.pages["Bitcoin"]

In [2]:
revs = list(page.revisions())

In [3]:
revs[0]

OrderedDict([('revid', 1208937822),
             ('parentid', 1208935636),
             ('user', 'A455bcd9'),
             ('timestamp',
              time.struct_time(tm_year=2024, tm_mon=2, tm_mday=19, tm_hour=16, tm_min=20, tm_sec=17, tm_wday=0, tm_yday=50, tm_isdst=-1)),
             ('comment',
              'Undid revision 1208935636 by [[Special:Contributions/Comingdeer|Comingdeer]] ([[User talk:Comingdeer|talk]]) Irrelevant here. Not encyclopedical.')])

In [4]:
revs = sorted(revs, key=lambda rev:rev["timestamp"])

In [5]:
revs[0]

OrderedDict([('revid', 275832581),
             ('parentid', 0),
             ('user', 'Pratyeka'),
             ('timestamp',
              time.struct_time(tm_year=2009, tm_mon=3, tm_mday=8, tm_hour=16, tm_min=41, tm_sec=7, tm_wday=6, tm_yday=67, tm_isdst=-1)),
             ('comment', 'creation (stub)')])

In [6]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

def find_sentiment(text):
    sent = sentiment_pipeline([text[:250]])[0]
    score = sent["score"]
    if sent["label"] == "NEGATIVE":
        score *= -1
    return score

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [8]:
edits = {}

for rev in revs:
    date = time.strftime("%Y-%m-%d", rev["timestamp"])

    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)

    edits[date]["edit_count"] += 1

    comment = rev.get("comment", "")
    edits[date]["sentiments"].append(find_sentiment(comment))

In [9]:
from statistics import mean

for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["sentiment"] = mean(edits[key]["sentiments"])
        edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
    else:
        edits[key]["sentiment"] = 0
        edits[key]["neg_sentiments"]  = 0

    del edits[key]["sentiments"]

In [11]:
import pandas as pd

edits_df = pd.DataFrame.from_dict(edits, orient="index")

In [13]:
edits_df

,edit_count,sentiment,neg_sentiment
2009-03-08,4,-0.550525,0.75
2009-08-05,1,0.748121,0.00
2009-08-06,2,0.995746,0.00
2009-08-14,1,0.930022,0.00
2009-10-13,2,-0.227499,0.50
...,...,...,...
2024-02-05,1,0.973620,0.00
2024-02-13,4,-0.504570,0.75
2024-02-15,1,-0.996920,1.00
2024-02-18,4,0.374312,0.25


In [14]:
edits_df.index = pd.to_datetime(edits_df.index)

In [15]:
from datetime import datetime

dates = pd.date_range(start="2009-03-08", end=datetime.today())

In [16]:
dates

DatetimeIndex(['2009-03-08', '2009-03-09', '2009-03-10', '2009-03-11',
               '2009-03-12', '2009-03-13', '2009-03-14', '2009-03-15',
               '2009-03-16', '2009-03-17',
               ...
               '2024-02-11', '2024-02-12', '2024-02-13', '2024-02-14',
               '2024-02-15', '2024-02-16', '2024-02-17', '2024-02-18',
               '2024-02-19', '2024-02-20'],
              dtype='datetime64[ns]', length=5463, freq='D')

In [17]:
edits_df = edits_df.reindex(dates, fill_value=0)

In [18]:
edits_df

,edit_count,sentiment,neg_sentiment
2009-03-08,4,-0.550525,0.75
2009-03-09,0,0.000000,0.00
2009-03-10,0,0.000000,0.00
2009-03-11,0,0.000000,0.00
2009-03-12,0,0.000000,0.00
...,...,...,...
2024-02-16,0,0.000000,0.00
2024-02-17,0,0.000000,0.00
2024-02-18,4,0.374312,0.25
2024-02-19,2,-0.989225,1.00


In [19]:
rolling_edits = edits_df.rolling(30).mean()

In [20]:
rolling_edits

,edit_count,sentiment,neg_sentiment
2009-03-08,NaN,NaN,NaN
2009-03-09,NaN,NaN,NaN
2009-03-10,NaN,NaN,NaN
2009-03-11,NaN,NaN,NaN
2009-03-12,NaN,NaN,NaN
...,...,...,...
2024-02-16,0.233333,0.015121,0.058333
2024-02-17,0.233333,0.015121,0.058333
2024-02-18,0.333333,-0.005119,0.066667
2024-02-19,0.400000,-0.038093,0.100000


In [21]:
rolling_edits = rolling_edits.dropna()

In [22]:
rolling_edits

,edit_count,sentiment,neg_sentiment
2009-04-06,0.133333,-0.018351,0.025000
2009-04-07,0.000000,0.000000,0.000000
2009-04-08,0.000000,0.000000,0.000000
2009-04-09,0.000000,0.000000,0.000000
2009-04-10,0.000000,0.000000,0.000000
...,...,...,...
2024-02-16,0.233333,0.015121,0.058333
2024-02-17,0.233333,0.015121,0.058333
2024-02-18,0.333333,-0.005119,0.066667
2024-02-19,0.400000,-0.038093,0.100000


In [23]:
rolling_edits.to_csv("wikipedia_edits.csv")

In [24]:
#next steps

In [ ]:
#scrape news articles
#scrape tweets
#scrape google trends 